# CMP 5006 - Information Security 


## Applications of Encryption


### Alejandro Proano, PhD. 

# Systems for Communication Integrity and Authentication

## Introduction

- **Integrity:** Ensuring messages are not altered during transmission
- **Authentication:** Verifying the identity of the sender
- Why they matter:
  - Prevent unauthorized modifications
  - Ensure message origin
  - Protect against man-in-the-middle attacks


## Cryptographic Primitives Review

- **Asymmetric Cryptography:** Public/Private key pairs
- **Symmetric Cryptography:** Shared secret keys
- **Cryptographic Hash Functions:** One-way, fixed-size output

```python
# Asymmetric example (simplified)
ciphertext = encrypt(public_key, plaintext)
plaintext = decrypt(private_key, ciphertext)

# Symmetric example
ciphertext = encrypt(shared_key, plaintext)
plaintext = decrypt(shared_key, ciphertext)

# Hash example
hash_value = hash("Hello, World!")
```



## Digital Signatures

- Uses asymmetric cryptography
- Provides integrity, authentication, and non-repudiation
- Process:
  1. Hash the message
  2. Encrypt hash with private key
  3. Verify with public key


### Digital Signatures

```python
def sign(private_key, message):
    message_hash = hash(message)
    return encrypt(private_key, message_hash)

def verify(public_key, message, signature):
    message_hash = hash(message)
    decrypted_hash = decrypt(public_key, signature)
    


## Message Authentication Codes (MACs)

- Uses symmetric cryptography
- Provides integrity and authentication
- Requires a shared secret key
- Process:
  1. Combine message and key
  2. Generate MAC
  3. Verify MAC with the same key


## MACs

```python
def generate_mac(key, message):
    return some_mac_function(key, message)

def verify_mac(key, message, mac):
    return generate_mac(key, message) == mac
```


## Hash-based Message Authentication Code (HMAC)

- Combines hash function and secret key
- More resistant to certain attacks than simple MACs
- Widely used in protocols like TLS


## HMACs

```python
def hmac(key, message):
    block_size = 64  # Example block size
    if len(key) > block_size:
        key = key
    if len(key) < block_size:
        key = key + b'\0' * (block_size - len(key))
    
    o_key_pad = bytes([0x5c ^ b for b in key])
    i_key_pad = bytes([0x36 ^ b for b in key])
    
    return hash(o_key_pad + hash(i_key_pad + message))
```


##  Digital Certificates

- Binds a public key to an entity's identity
- Issued and signed by trusted Certificate Authorities (CAs)
- Contains:
  - Subject's information
  - Public key
  - Issuer information
  - Digital signature of the CA


## Digital Certificates

```python
certificate = {
    "subject": "example.com",
    "public_key": "...",
    "issuer": "Trusted CA",
    "valid_from": "2023-01-01",
    "valid_to": "2024-01-01",
    "signature": "..."
}
```


##  Practical Applications and Designs


## Secure Email System

```python
def send_secure_email(sender_private_key, recipient_public_key, message):
    signature = sign(sender_private_key, message)
    encrypted_message = encrypt(recipient_public_key, message + signature)
    return encrypted_message

def receive_secure_email(recipient_private_key, sender_public_key, encrypted_message):
    decrypted_content = decrypt(recipient_private_key, encrypted_message)
    message, signature = split_message_and_signature(decrypted_content)
    if verify(sender_public_key, message, signature):
        return message
    else:
        raise Exception("Invalid signature")
```


## Secure API Communication

```python
def api_request(client_id, shared_secret, payload):
    timestamp = get_current_timestamp()
    message = f"{client_id}:{timestamp}:{payload}"
    hmac_signature = hmac(shared_secret, message)
    
    headers = {
        "Authorization": f"HMAC {client_id}:{timestamp}:{hmac_signature}"
    }
    
    return send_http_request(payload, headers)

def verify_api_request(request, shared_secrets):
    auth_header = request.headers["Authorization"]
    client_id, timestamp, signature = parse_auth_header(auth_header)
    
    if is_timestamp_valid(timestamp) and client_id in shared_secrets:
        expected_signature = hmac(shared_secrets[client_id], f"{client_id}:{timestamp}:{request.body}")
        return signature == expected_signature
    
    return False
```


# Sharing Secret Keys for Symmetric Cryptography


## Introduction

- Symmetric cryptography relies on shared secret keys
- Challenge: Securely distributing keys to communicating parties
- Importance of secure key exchange:
  - Prevents unauthorized access to encrypted communications
  - Ensures confidentiality and integrity of data


## The Key Distribution Problem

- How to securely share a secret key over an insecure channel?
- Vulnerabilities:
  - Eavesdropping
  - Man-in-the-middle attacks
  - Impersonation


## Diffie-Hellman Key Exchange

- Allows two parties to generate a shared secret over an insecure channel
- Based on the discrete logarithm problem

```python
# Simplified Diffie-Hellman example
p = 23  # Prime modulus
g = 5   # Generator

# Alice
a = 6  # Alice's private key
A = (g ** a) % p  # Alice's public key

# Bob
b = 15  # Bob's private key
B = (g ** b) % p  # Bob's public key

# Shared secret calculation
s_alice = (B ** a) % p
s_bob = (A ** b) % p

assert s_alice == s_bob  # Shared secret is the same for both
```


## Key Distribution Center (KDC)

- Trusted third party that distributes keys to authenticated users
- Example: Kerberos authentication protocol

```
1. Client -> KDC: Request ticket
2. KDC -> Client: Ticket encrypted with client's key
3. Client -> Server: Ticket + Authenticator
4. Server -> Client: Session key confirmation
```

Advantages:
- Centralized key management
- Supports authentication

Disadvantages:
- Single point of failure
- Scalability issues


## Public Key Infrastructure (PKI)

- Uses asymmetric cryptography to establish symmetric keys
- Relies on trusted Certificate Authorities (CAs)

Process:
1. Client obtains server's public key (certificate)
2. Client generates symmetric key
3. Client encrypts symmetric key with server's public key
4. Server decrypts symmetric key with its private key

```python
# Simplified example
server_public_key = get_server_certificate().public_key
symmetric_key = generate_random_key()
encrypted_symmetric_key = encrypt(server_public_key, symmetric_key)
send_to_server(encrypted_symmetric_key)
```


## Out-of-Band Key Exchange

- Exchange keys through a separate, secure channel
- Examples:
  - In-person meeting
  - Secure phone call
  - Physical mail

Advantages:
- Simple and secure (if done correctly)
- No reliance on complex protocols or third parties

Disadvantages:
- Not scalable
- Impractical for many scenarios


## Forward Secrecy

- Ensures past communications remain secure if long-term keys are compromised
- Achieved by generating ephemeral session keys

### Diffie-Hellman Ephemeral (DHE)

- Generates new Diffie-Hellman parameters for each session
- Used in protocols like TLS

```python
def generate_dhe_keys():
    private_key = generate_random_number()
    public_key = (g ** private_key) % p
    return private_key, public_key

# For each new session
alice_private, alice_public = generate_dhe_keys()
bob_private, bob_public = generate_dhe_keys()

shared_secret = (bob_public ** alice_private) % p
```
